In [1]:
## https://www.cnblogs.com/luohenyueji/p/17725774.html#21-%E5%9F%BA%E4%BA%8E%E8%A7%84%E5%88%99%E7%9A%84%E4%B8%AD%E6%96%87%E6%96%87%E6%9C%AC%E7%BA%A0%E9%94%99
## ***windows 安装失败，建议 基于linux 尝试***
##[自然语言处理] 基于pycorrector实现文本纠错 pip install -U pycorrector
### 下载语言模型： https://deepspeech.bj.bcebos.com/zh_lm/zh_giga.no_cna_cmn.prune01244.klm
### 放到目录： /home/user/.pycorrector/datasets 下即可使用
import warnings
warnings.filterwarnings("ignore")


In [2]:
##文本纠错

import pycorrector
# include_symbol: 是否包含标点符号，默认为True
# threshold: 纠错阈值，默认为57
corrected_sent, detail = pycorrector.correct("人群穿流不息，少先队员因该为老人让坐!",include_symbol=True,threshold=60)
# corrected_sent: 纠错后的句子
# detail: 纠错信息，[wrong, right, begin_pos, end_pos]
print(corrected_sent, detail)

#####2023-10-11 15:37:55.092 | DEBUG    | pycorrector.detector:_initialize_detector:89 - Loaded language model: /home/user/.pycorrector/datasets/zh_giga.no_cna_cmn.prune01244.klm
#####人群川流不息，少先队员应该为老人让座! [('穿流不息', '川流不息', 2, 6), ('因该', '应该', 11, 13), ('坐', '座', 17, 18)]
#####

2023-10-11 16:44:00.181 | DEBUG    | pycorrector.detector:_initialize_detector:89 - Loaded language model: /home/user/.pycorrector/datasets/zh_giga.no_cna_cmn.prune01244.klm


人群川流不息，少先队员应该为老人让座! [('穿流不息', '川流不息', 2, 6), ('因该', '应该', 11, 13), ('坐', '座', 17, 18)]


In [3]:
##错误检测
import pycorrector
 
idx_errors = pycorrector.detect('人群穿流不息，少先队员因该为老人让坐!')
print(idx_errors)
##[['穿流不息', 2, 6, 'proper'], ['因该', 11, 13, 'word'], ['坐', 17, 18, 'char']]


[['穿流不息', 2, 6, 'proper'], ['因该', 11, 13, 'word'], ['坐', 17, 18, 'char']]


In [4]:
##成语、专名纠错
from pycorrector.proper_corrector import ProperCorrector
from pycorrector import config
 
m = ProperCorrector(proper_name_path=config.proper_name_path)
x = [
    '这块名表带带相传',
    '这块名表代代相传',
    '这场比赛我甘败下风',
    '这场比赛我甘拜下封',
    '早上在拼哆哆上买了点葡桃',
]
 
for i in x:
    print(i, ' -> ', m.proper_correct(i))

###
###
###这块名表带带相传  ->  ('这块名表代代相传', [('带带相传', '代代相传', 4, 8)])
###这块名表代代相传  ->  ('这块名表代代相传', [])
###这场比赛我甘败下风  ->  ('这场比赛我甘拜下风', [('甘败下风', '甘拜下风', 5, 9)])
###这场比赛我甘拜下封  ->  ('这场比赛我甘拜下风', [('甘拜下封', '甘拜下风', 5, 9)])
###早上在拼哆哆上买了点葡桃  ->  ('早上在拼多多上买了点葡桃', [('拼哆哆', '拼多多', 3, 6)])


这块名表带带相传  ->  ('这块名表代代相传', [('带带相传', '代代相传', 4, 8)])
这块名表代代相传  ->  ('这块名表代代相传', [])
这场比赛我甘败下风  ->  ('这场比赛我甘拜下风', [('甘败下风', '甘拜下风', 5, 9)])
这场比赛我甘拜下封  ->  ('这场比赛我甘拜下风', [('甘拜下封', '甘拜下风', 5, 9)])
早上在拼哆哆上买了点葡桃  ->  ('早上在拼多多上买了点葡桃', [('拼哆哆', '拼多多', 3, 6)])


In [5]:
###自定义混淆集

from pycorrector import ConfusionCorrector, Corrector
 
if __name__ == '__main__':
    error_sentences = [
        '买iphonex，要多少钱',  # 漏召回
        '哪里卖苹果吧？请大叔给我让坐',  # 漏召回
        '共同实际控制人萧华、霍荣铨、张旗康',  # 误杀
        '上述承诺内容系本人真实意思表示',  # 正常
        '大家一哄而伞怎么回事',  # 成语
    ]
    m = Corrector()
    for i in error_sentences:
        print(i, ' -> ', m.detect(i), m.correct(i))
 
    print('*' * 42)
    
    # 自定义混淆集
    custom_confusion = {'得事': '的事', '天地无垠': '天地无限', '交通先行': '交通限行', '苹果吧': '苹果八', 'iphonex': 'iphoneX', '小明同学': '小茗同学', '萧华': '萧华',
                        '张旗康': '张旗康', '一哄而伞': '一哄而散', 'happt': 'happen', 'shylock': 'shylock', '份额': '份额', '天俺门': '天安门'}
    m = ConfusionCorrector(custom_confusion_path_or_dict=custom_confusion)
    for i in error_sentences:
        print(i, ' -> ', m.confusion_correct(i))


###买iphonex，要多少钱  ->  [['钱', 12, 13, 'char']] ('买iphonex，要多少钱', [])
###**哪里卖苹果吧？请大叔给我让坐  ->  [] ('哪里卖苹果吧？请大叔给我让坐', [])
###**共同实际控制人萧华、霍荣铨、张旗康  ->  [['霍荣铨', 10, 13, 'word'], ['张旗康', 14, 17, 'word']] ('共同实际控制人萧华、霍荣铨、张启康', [('张旗康', '张启康', 14, 17)])
###**上述承诺内容系本人真实意思表示  ->  [['系', 6, 7, 'char']] ('上述承诺内容系本人真实意思表示', [])
###**大家一哄而伞怎么回事  ->  [['一哄', 2, 4, 'word'], ['伞', 5, 6, 'char']] ('大家一哄而散怎么回事', [('伞', '散', 5, 6)])
###********************************************
###**买iphonex，要多少钱  ->  ('买iphoneX，要多少钱', [['iphonex', 'iphoneX', 1, 8]])
###**哪里卖苹果吧？请大叔给我让坐  ->  ('哪里卖苹果八？请大叔给我让坐', [['苹果吧', '苹果八', 3, 6]])
###**共同实际控制人萧华、霍荣铨、张旗康  ->  ('共同实际控制人萧华、霍荣铨、张旗康', [['萧华', '萧华', 7, 9], ['张旗康', '张旗康', 14, 17]])
###**上述承诺内容系本人真实意思表示  ->  ('上述承诺内容系本人真实意思表示', [])
###**大家一哄而伞怎么回事  ->  ('大家一哄而散怎么回事', [['一哄而伞', '一哄而散', 2, 6]])


2023-10-11 16:44:17.602 | DEBUG    | pycorrector.detector:_initialize_detector:89 - Loaded language model: /home/user/.pycorrector/datasets/zh_giga.no_cna_cmn.prune01244.klm


买iphonex，要多少钱  ->  [['钱', 12, 13, 'char']] ('买iphonex，要多少钱', [])
哪里卖苹果吧？请大叔给我让坐  ->  [] ('哪里卖苹果吧？请大叔给我让坐', [])
共同实际控制人萧华、霍荣铨、张旗康  ->  [['霍荣铨', 10, 13, 'word'], ['张旗康', 14, 17, 'word']] ('共同实际控制人萧华、霍荣铨、张启康', [('张旗康', '张启康', 14, 17)])
上述承诺内容系本人真实意思表示  ->  [['系', 6, 7, 'char']] ('上述承诺内容系本人真实意思表示', [])


2023-10-11 16:44:21.707 | DEBUG    | pycorrector.confusion_corrector:__init__:22 - Loaded confusion size: 13


大家一哄而伞怎么回事  ->  [['一哄', 2, 4, 'word'], ['伞', 5, 6, 'char']] ('大家一哄而散怎么回事', [('伞', '散', 5, 6)])
******************************************
买iphonex，要多少钱  ->  ('买iphoneX，要多少钱', [['iphonex', 'iphoneX', 1, 8]])
哪里卖苹果吧？请大叔给我让坐  ->  ('哪里卖苹果八？请大叔给我让坐', [['苹果吧', '苹果八', 3, 6]])
共同实际控制人萧华、霍荣铨、张旗康  ->  ('共同实际控制人萧华、霍荣铨、张旗康', [['萧华', '萧华', 7, 9], ['张旗康', '张旗康', 14, 17]])
上述承诺内容系本人真实意思表示  ->  ('上述承诺内容系本人真实意思表示', [])
大家一哄而伞怎么回事  ->  ('大家一哄而散怎么回事', [['一哄而伞', '一哄而散', 2, 6]])


In [ ]:
##自定义语言模型
# 自定义模型路径
lm_path = './custom.klm'
model = Corrector(language_model_path=lm_path)


In [6]:
###英文拼写纠错
sent = "what happending? how to speling it, can you gorrect it?"
corrected_text, details = pycorrector.en_correct(sent)
print(sent, '=>', corrected_text)
print(details)

2023-10-11 16:44:24.958 | DEBUG    | pycorrector.en_spell:_init:39 - load en spell data: /home/user/.local/lib/python3.8/site-packages/pycorrector/data/en/en.json.gz, size: 30120


what happending? how to speling it, can you gorrect it? => what happening? how to spelling it, can you correct it?
[('happending', 'happening', 5, 15), ('speling', 'spelling', 24, 31), ('gorrect', 'correct', 44, 51)]


In [7]:
from pycorrector.en_spell import EnSpell
 
# # 定义一个字符串变量
sent = "what is your name? shylock?"  
# 创建一个EnSpell类的实例对象
spell = EnSpell()  
corrected_text, details = spell.correct(sent) 
# shylock被纠错为shock
print(sent, '=>', corrected_text, details) 
print('-' * 42) 
 
# 定义一个包含词频信息的字典
# 设置shylock出现频次比shock高
my_dict = {'your': 120, 'name': 2, 'is': 1, 'shock': 2, 'shylock': 1, 'what': 1} 
# 创建一个EnSpell类的实例对象，并传入自定义词频字典
spell = EnSpell(word_freq_dict=my_dict)  
corrected_text, details = spell.correct(sent)  
print(sent, '=>', corrected_text, details)  

2023-10-11 16:44:29.785 | DEBUG    | pycorrector.en_spell:_init:39 - load en spell data: /home/user/.local/lib/python3.8/site-packages/pycorrector/data/en/en.json.gz, size: 30120


what is your name? shylock? => what is your name? shock? [('shylock', 'shock', 19, 26)]
------------------------------------------
what is your name? shylock? => what is your name? shylock? []


In [8]:
###中文简繁互换
import pycorrector
 
traditional_sentence = '學而時習之，不亦說乎'
simplified_sentence = pycorrector.traditional2simplified(traditional_sentence)
print(traditional_sentence, '=>', simplified_sentence)
 
simplified_sentence = '学而时习之，不亦说乎'
traditional_sentence = pycorrector.simplified2traditional(simplified_sentence)
print(simplified_sentence, '=>', traditional_sentence)

學而時習之，不亦說乎 => 学而时习之，不亦说乎
学而时习之，不亦说乎 => 學而時習之，不亦說乎


In [ ]:
pycorrector提供多个基于深度学习的中文文本纠错模型，一般而言，使用深度学习进行中文文本纠错可以获得比基于规则纠错更好的效果。pycorrector在SIGHAN2015数据集数据集下对各种深度学习模型进行了评测，SIGHAN2015数据集是一个经典公开的用于中文文本纠错任务的数据集，并得出以下结论：

中文拼写纠错模型效果最好的是MacBert-CSC，模型名称是shibing624/macbert4csc-base-chinese，huggingface model：shibing624/macbert4csc-base-chinese
中文语法纠错模型效果最好的是BART-CSC，模型名称是shibing624/bart4csc-base-chinese，huggingface model：shibing624/bart4csc-base-chinese
最具潜力的模型是Mengzi-T5-CSC，模型名称是shibing624/mengzi-t5-base-chinese-correction，huggingface model：shibing624/mengzi-t5-base-chinese-correction，未改变模型结构，仅fine-tune中文纠错数据集，已经在SIGHAN 2015取得接近SOTA的效果
基于ChatGLM-6B的纠错微调模型效果也不错，模型名称是shibing624/chatglm-6b-csc-zh-lora，huggingface model：shibing624/chatglm-6b-csc-zh-lora，大模型不仅能改错还能润色句子，但是模型太大，推理速度慢
在pycorrector中调用MacBert-CSC模型进行文本纠错的代码如下，该代码将自动加载macbert4csc-base-chinese提供的纠错模型。

In [ ]:
##pip3 install transformers

In [ ]:
###基于深度学习 纠错模型 推荐： git clone https://huggingface.co/shibing624/macbert4csc-base-chinese
from pycorrector.macbert.macbert_corrector import MacBertCorrector
from pycorrector import ConfusionCorrector
 
if __name__ == '__main__':
    error_sentences = [
        '少先队员因该为老人让坐',
        '机七学习是人工智能领遇最能体现智能的一个分知',
    ]
 
    m = MacBertCorrector()
    # add confusion corrector for postprocess
    confusion_dict = {"喝小明同学": "喝小茗同学", "老人让坐": "老人让座", "平净": "平静", "分知": "分支"}
    cm = ConfusionCorrector(custom_confusion_path_or_dict=confusion_dict)
    for line in error_sentences:
        correct_sent, err = m.macbert_correct(line)
        print("query:{} => {} err:{}".format(line, correct_sent, err))
        correct_sent, err = cm.confusion_correct(correct_sent)
        if err:
            print("added confusion: {} err: {}".format(correct_sent, err))